## Imported packages

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import xml.etree.cElementTree as et
import requests
import time
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display

## 2017 Price cleaning

Price from 2017 was first imported in

In [11]:
price2017 = pd.read_csv('../data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

as data was relatively cleaned in a similar way, a function was defined to ensure that the full address in the data was correctly formatted to query the SLA API.

In [12]:
def cleaning(df, outputfilename):
    """
    Cleans the files into usable format for querying latitude and longitude
    
    Parameters
    ----------
    df: dataframe containing data to be cleaned
    outputfilename: name of the csv file to be saved (in string)
    
    Returns
    -------
    csv file named with outputfilename containing dataframe data
    """
    #making new address with block and street_name
    df['full_add'] = df['block']+" " + df['street_name']
    #cleaning the 'st.' to 'saint'
    df['full_add'] = [x.replace('ST.', 'SAINT') for x in df['full_add']]
    
    #printing out csv file
    df.to_csv(('../data/'+outputfilename+'.csv'))

In [13]:
cleaning(price, "price")

### Querying API to get lat and longitude coordinates

API query was scraped using the latlongscraper.py in repo.

In [15]:
price = pd.read_csv('../data/priceonly.csv')

### getting routing info

Route info was scraped by scraper_partial.py file in repo

### combining lat and long with price dataframes

In [16]:
priced1 = pd.read_csv('../data/priced_1.csv').T
priced2 = pd.read_csv('../data/priced_2.csv').T
priced3 = pd.read_csv('../data/priced_3.csv').T
priced4 = pd.read_csv('../data/priced_4.csv').T
priced5 = pd.read_csv('../data/priced_5.csv').T
priced6 = pd.read_csv('../data/priced_6.csv').T
priced7 = pd.read_csv('../data/priced_7.csv').T
priced8 = pd.read_csv('../data/priced_8.csv').T

In [17]:
price_latlong_list = [priced1, priced2, priced3, priced4, priced5, priced6, priced7, priced8]

In [18]:
def lat_long_add(csvlist, df):
    """
    adds all the segments of file
    Parameters
    ----------
    csvlist: list of csvs to join 
    df: dataframe to add to
    
    Output
    ------
    updated df
    """
    temp = pd.DataFrame(columns = ['Unnamed: 0', 'rafflestime'])
    add = {}

    for x in csvlist: 
        for i in range(1, x.shape[0]):
            add[x.index[i]] = x.iloc[i][0]
            
    #rafflestime = []
    #for x in df['full_add']:
    #    try:
    #        rafflestime.append(add[x])
    #    except: 
    #        rafflestime.append("")
    df['rafflestime'] = [add[x] for x in df['full_add']]
        

In [19]:
lat_long_add(price_latlong_list, price)

In [20]:
price.drop("Unnamed: 0",axis = 1, inplace = True)

In [21]:
price.to_csv("../data/cleaned2017price.csv")

## 2015 Price cleaning

In [23]:
price2015 = pd.read_csv('../data/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')


In [24]:
cleaning(price2015, "2015price")

### Querying API to get lat and longitude coordinates

API query was scraped using the latlongscraper.py in repo.

In [153]:
old_latlong1 = pd.read_csv('../data/2015latlong1.csv').T
old_latlong2 = pd.read_csv('../data/2015latlong2.csv').T
oldlatlonglist = [old_latlong1, old_latlong2]

In [154]:
old_latlong = pd.DataFrame(columns = ['full_add', 'lat', 'long', 'latlong'])

In [155]:
def combinerlatlong(dflist, resultdf):
    """
    combines queried lat and long files from full_add
    Parameters
    ----------
    dflist: list of dataframes imported (list can only contain 2 dataframes only)
    resultdf: empty dataframe for results to fill in (must contain expected columns)
    
    Returns
    -------
    resultdf: completed joined dataframe
    """
    for x in dflist: 
        x.columns = x.iloc[0]
        x.drop(x.index[0], inplace= True)
        resultdf = resultdf.append(x)
    resultdf.reset_index(drop= True, inplace = True)
    return resultdf

In [156]:
old_latlong = combinerlatlong(oldlatlonglist, old_latlong)

In [157]:
old_latlong

Unnamed: 0,full_add,lat,long,latlong
0,174 ANG MO KIO AVE 4,1.37509746867904,103.83761896123,"1.37509746867904,103.83761896123"
1,541 ANG MO KIO AVE 10,1.37392239168829,103.855621371068,"1.37392239168829,103.855621371068"
2,163 ANG MO KIO AVE 4,1.3735485391993,103.83817647139799,"1.3735485391993,103.83817647139799"
3,446 ANG MO KIO AVE 10,1.3677609513095599,103.85535714590799,"1.3677609513095599,103.85535714590799"
4,557 ANG MO KIO AVE 10,1.37162570203323,103.857736107527,"1.37162570203323,103.857736107527"
...,...,...,...,...
13659,654 YISHUN AVE 4,1.42305346445826,103.840410313172,"1.42305346445826,103.840410313172"
13660,795 YISHUN RING RD,1.4196943122303602,103.83255122685,"1.4196943122303602,103.83255122685"
13661,386 YISHUN RING RD,1.42898064926519,103.846473489722,"1.42898064926519,103.846473489722"
13662,664 YISHUN AVE 4,1.42019986585644,103.84107464313601,"1.42019986585644,103.84107464313601"


In [162]:
old_latlong_dict1 = {}

In [163]:
def latlongtodict(df, resultdict, pricedf):
    """
    converts latlong dict to a dictionary, and then creates a new column 
    for latitude and longitude on the dataframe that contains price
    
    Parameters
    ----------
    df: dataframe that contains the queried address and latitude and longitude
    resultdict: empty dictionary that will contain the results of the function
    pricedf: corresponding dataframe containing price data
    
    Returns
    -------
    resultdf: updated dictionary containing results
    
    """
    for i in range(len(df['full_add'])):
        key = df['full_add'][i]
        value = df['latlong'][i]
        resultdict[key] = value
    pricedf['latlong'] = [resultdict[x] for x in pricedf['full_add']]
    return resultdict

In [164]:
old_latlong_dict1 = latlongtodict(old_latlong, old_latlong_dict1, price2015)

In [166]:
price2015.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,full_add,latlong
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4,"1.37509746867904,103.83761896123"
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,541 ANG MO KIO AVE 10,"1.37392239168829,103.855621371068"
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,163 ANG MO KIO AVE 4,"1.3735485391993,103.83817647139799"
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,446 ANG MO KIO AVE 10,"1.3677609513095599,103.85535714590799"
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,557 ANG MO KIO AVE 10,"1.37162570203323,103.857736107527"


In [167]:
price2015.to_csv('../data/2015pricewlatlong.csv')

### getting routing info

Route info was scraped by scraper_partial.py file in repo

In [168]:
oldpriced1 = pd.read_csv('../data/oldpriced_1.csv').T
oldpriced2 = pd.read_csv('../data/oldpriced_2.csv').T
oldpriced3 = pd.read_csv('../data/oldpriced_3.csv').T
oldpriced4 = pd.read_csv('../data/oldpriced_4.csv').T
oldpriced5 = pd.read_csv('../data/oldpriced_5.csv').T
oldpriced6 = pd.read_csv('../data/oldpriced_6.csv').T

In [169]:
oldpriced_list = [oldpriced1, oldpriced2, oldpriced3, oldpriced4, oldpriced5, oldpriced6]

In [170]:
oldprice = pd.read_csv('../data/2015pricewlatlong.csv')

In [171]:
lat_long_add(oldpriced_list, oldprice)

In [172]:
oldprice.head()

,Unnamed: 0,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,full_add,latlong,rafflestime
0,0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4,"1.37509746867904,103.83761896123",2006.0
1,1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,541 ANG MO KIO AVE 10,"1.37392239168829,103.855621371068",2152.0
2,2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,163 ANG MO KIO AVE 4,"1.3735485391993,103.83817647139799",2018.0
3,3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,446 ANG MO KIO AVE 10,"1.3677609513095599,103.85535714590799",1636.0
4,4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,557 ANG MO KIO AVE 10,"1.37162570203323,103.857736107527",1851.0


In [144]:
oldprice.to_csv('../data/cleaned2015price.csv')

### 2012 price cleaning

In [174]:
price2012 = pd.read_csv('../data/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')

In [175]:
cleaning(price2012, "2012price")

### Querying API to get lat and longitude coordinates

API query was scraped using the latlongscraper.py in repo.

In [176]:
older_latlong1 = pd.read_csv('../data/2012latlong1.csv').T
older_latlong2 = pd.read_csv('../data/2012latlong2.csv').T
olderlatlonglist = [older_latlong1, older_latlong2]

In [177]:
older_latlong = pd.DataFrame(columns = ['full_add', 'lat', 'long', 'latlong'])

In [178]:
older_latlong = combinerlatlong(olderlatlonglist, older_latlong)

In [179]:
older_latlong

Unnamed: 0,full_add,lat,long,latlong
0,172 ANG MO KIO AVE 4,1.37477684251468,103.836707903222,"1.37477684251468,103.836707903222"
1,510 ANG MO KIO AVE 8,1.37340092645027,103.849073244454,"1.37340092645027,103.849073244454"
2,610 ANG MO KIO AVE 4,1.37939470957041,103.83915723132601,"1.37939470957041,103.83915723132601"
3,474 ANG MO KIO AVE 10,1.36275784702216,103.85801532366699,"1.36275784702216,103.85801532366699"
4,604 ANG MO KIO AVE 5,1.37986658053968,103.835976650638,"1.37986658053968,103.835976650638"
...,...,...,...,...
14550,777 YISHUN AVE 2,1.42201001071953,103.83433309610201,"1.42201001071953,103.83433309610201"
14551,711 YISHUN AVE 5,1.4280012028918898,103.827147006794,"1.4280012028918898,103.827147006794"
14552,612 YISHUN ST 61,1.4203607394506002,103.835417557256,"1.4203607394506002,103.835417557256"
14553,819 YISHUN ST 81,1.4125308542767598,103.83422787719499,"1.4125308542767598,103.83422787719499"


In [180]:
older_latlong_dict1 = {}

In [181]:
older_latlong_dict1 = latlongtodict(older_latlong, older_latlong_dict1, price2012)

In [182]:
price2012.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,full_add,latlong
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,172 ANG MO KIO AVE 4,"1.37477684251468,103.836707903222"
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,510 ANG MO KIO AVE 8,"1.37340092645027,103.849073244454"
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,610 ANG MO KIO AVE 4,"1.37939470957041,103.83915723132601"
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,474 ANG MO KIO AVE 10,"1.36275784702216,103.85801532366699"
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,604 ANG MO KIO AVE 5,"1.37986658053968,103.835976650638"


In [183]:
price2012.to_csv('../data/2012pricewlatlong.csv')

### getting routing info

Route info was scraped by scraper_partial.py file in repo

In [184]:
olderpriced1 = pd.read_csv('../data/olderpriced_1.csv').T
olderpriced2 = pd.read_csv('../data/olderpriced_2.csv').T
olderpriced3 = pd.read_csv('../data/olderpriced_3.csv').T
olderpriced4 = pd.read_csv('../data/olderpriced_4.csv').T
olderpriced5 = pd.read_csv('../data/olderpriced_5.csv').T
olderpriced6 = pd.read_csv('../data/olderpriced_6.csv').T
olderpriced7 = pd.read_csv('../data/olderpriced_7.csv').T
olderpriced8 = pd.read_csv('../data/olderpriced_8.csv').T

In [186]:
olderpriced_list = [olderpriced1, olderpriced2, olderpriced3, 
                  olderpriced4, olderpriced5, olderpriced6, 
                  olderpriced7, olderpriced8]

In [187]:
olderprice = pd.read_csv('../data/2012pricewlatlong.csv')

In [188]:
lat_long_add(olderpriced_list, olderprice)

In [189]:
olderprice.head()

,Unnamed: 0,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,full_add,latlong,rafflestime
0,0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,172 ANG MO KIO AVE 4,"1.37477684251468,103.836707903222",1983.0
1,1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,510 ANG MO KIO AVE 8,"1.37340092645027,103.849073244454",1616.0
2,2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,610 ANG MO KIO AVE 4,"1.37939470957041,103.83915723132601",2020.0
3,3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,474 ANG MO KIO AVE 10,"1.36275784702216,103.85801532366699",NaN
4,4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,604 ANG MO KIO AVE 5,"1.37986658053968,103.835976650638",1931.0


In [190]:
olderprice.to_csv('../data/cleaned2012price.csv')

## getting bus stop data

In [28]:
### scraping bus stops
busurl = 'http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip='

def get_bus_stops(url):
    count = 0
    busstop = {}
    headers = {'AccountKey' : 'rz6BtGMwQLGn8O5yoUwunA==',
               'accept' : 'application/json'}
    while count<5500:
        new_url = url+str(count)
        print(new_url)
        busstopget = requests.get(new_url, headers = headers)
        print(busstopget)
        if busstopget.status_code == 200:
            result = busstopget.json()
            for x in result['value']:
                key = x['BusStopCode']
                description = x['Description']
                lat = x['Latitude']
                long = x['Longitude']
                busstop[key] = {'Bus Stop Code': key, 'Description' : description, 'lat' : lat, 'long': long}
            count += 500
        else:
            count += 500
    df = pd.DataFrame(busstop).T
    df.reset_index(inplace = True)
    print(df)
    df.to_csv('../data/busstopdate.csv', index = True)

get_bus_stops(busurl)

http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=0
<Response [200]>
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=500
<Response [200]>
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=1000
<Response [200]>
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=1500
<Response [200]>
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=2000
<Response [200]>
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=2500
<Response [200]>
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=3000
<Response [200]>
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=3500
<Response [200]>
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=4000
<Response [200]>
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=4500
<Response [200]>
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=5000
<Response [200]>
      index Bus Stop Code                 Description      lat     lo

## Cleaning school data

In [193]:
school = pd.read_csv('../data/general-information-of-schools.csv')
school.drop(labels = ['url_address', 'email_address', 'principal_name',
                      'first_vp_name', 'second_vp_name', 'third_vp_name',
                      'fourth_vp_name', 'fifth_vp_name', 'sixth_vp_name',
                      'visionstatement_desc', 'missionstatement_desc',
                      'philosophy_culture_ethos', 'sap_ind', 'autonomous_ind',
                      'gifted_ind', 'ip_ind', 'mothertongue1_code', 
                      'mothertongue2_code','mothertongue3_code', 'zone_code',
                      'cluster_code', 'type_code', 'session_code',
                      'telephone_no', 'telephone_no_2', 'fax_no', 'fax_no_2',
                      'special_sdp_offered'], axis = 1, inplace = True)

In [194]:
school.columns

school.to_csv('../data/school.csv')

In [219]:
school.columns = ['school_name', 'full_add', 'postal_code',
                  'mrt_desc', 'bus_desc','dgp_code',
                  'nature_code', 'mainlevel_code']


In [221]:
school.head()

,school_name,full_add,postal_code,mrt_desc,bus_desc,dgp_code,nature_code,mainlevel_code
0,NATIONAL JUNIOR COLLEGE,37 HILLCREST ROAD,288913,"BOTANIC GARDENS MRT, TAN KAH KEE MRT, SIXTH AV...","151, 154, 156, 157, 170, 170X, 171, 174, 66, 6...",BUKIT TIMAH,CO-ED SCHOOL,MIXED LEVEL
1,TEMASEK JUNIOR COLLEGE,22 BEDOK SOUTH ROAD,469278,BEDOK MRT,"12, 16, 31, 38, 47, 137, 155, 196, 197, 229",BEDOK,CO-ED SCHOOL,MIXED LEVEL
2,VICTORIA JUNIOR COLLEGE,20 MARINE VISTA,449035,"Nearest MRT Stations: Kembangan, Eunos","13, 16, 31, 36, 43, 48, 55, 135, 196, 196e, 197",BEDOK,CO-ED SCHOOL,JUNIOR COLLEGE
3,EUNOIA JUNIOR COLLEGE,53 MOUNT SINAI ROAD,276880,"Buona Vista MRT Station, Dover MRT Station","92, 100, 105, 106, 111, 147, 185",BISHAN,CO-ED SCHOOL,JUNIOR COLLEGE
4,ANDERSON SERANGOON JUNIOR COLLEGE,4500 ANG MO KIO AVENUE 6,569843,Yio Chu Kang MRT Station,"13, 45, 86, 162, 269, 825, 851, 852, 853",ANG MO KIO,CO-ED SCHOOL,JUNIOR COLLEGE


### School latitude and longitude
School Latitude and Longitude was queried

In [209]:
schoollatlong = pd.read_csv('../data/school_latlong.csv').T
schoollatlong.head()

,0,1,2,3
Unnamed: 0,full_add,lat,long,latlong
37 HILLCREST ROAD,37 HILLCREST ROAD,1.33042992800757,103.804458713785,"1.33042992800757,103.804458713785"
22 BEDOK SOUTH ROAD,22 BEDOK SOUTH ROAD,1.31846190599677,103.93559797008899,"1.31846190599677,103.93559797008899"
20 MARINE VISTA,20 MARINE VISTA,1.3057179654943,103.919531033331,"1.3057179654943,103.919531033331"
53 MOUNT SINAI ROAD,53 MOUNT SINAI ROAD,1.3137449647427601,103.786441132957,"1.3137449647427601,103.786441132957"


In [210]:
schoollatlong.columns = schoollatlong.iloc[0]
schoollatlong.drop(schoollatlong.index[0], inplace= True)

schoollatlong.head()

Unnamed: 0,full_add,lat,long,latlong
37 HILLCREST ROAD,37 HILLCREST ROAD,1.33042992800757,103.804458713785,"1.33042992800757,103.804458713785"
22 BEDOK SOUTH ROAD,22 BEDOK SOUTH ROAD,1.31846190599677,103.93559797008899,"1.31846190599677,103.93559797008899"
20 MARINE VISTA,20 MARINE VISTA,1.3057179654943,103.919531033331,"1.3057179654943,103.919531033331"
53 MOUNT SINAI ROAD,53 MOUNT SINAI ROAD,1.3137449647427601,103.786441132957,"1.3137449647427601,103.786441132957"
4500 ANG MO KIO AVENUE 6,4500 ANG MO KIO AVENUE 6,1.37839670972544,103.845885968342,"1.37839670972544,103.845885968342"


In [211]:
schoollatlong.reset_index(drop= True, inplace = True)
schoollatlong.head()

Unnamed: 0,full_add,lat,long,latlong
0,37 HILLCREST ROAD,1.33042992800757,103.804458713785,"1.33042992800757,103.804458713785"
1,22 BEDOK SOUTH ROAD,1.31846190599677,103.93559797008899,"1.31846190599677,103.93559797008899"
2,20 MARINE VISTA,1.3057179654943,103.919531033331,"1.3057179654943,103.919531033331"
3,53 MOUNT SINAI ROAD,1.3137449647427601,103.786441132957,"1.3137449647427601,103.786441132957"
4,4500 ANG MO KIO AVENUE 6,1.37839670972544,103.845885968342,"1.37839670972544,103.845885968342"


In [213]:
schoollatlong.columns

Index(['full_add', 'lat', 'long', 'latlong'], dtype='object', name='Unnamed: 0')

In [222]:
school_dict = {}
school_dict = latlongtodict(schoollatlong, school_dict, school)

In [223]:
school.head()

,school_name,full_add,postal_code,mrt_desc,bus_desc,dgp_code,nature_code,mainlevel_code,latlong
0,NATIONAL JUNIOR COLLEGE,37 HILLCREST ROAD,288913,"BOTANIC GARDENS MRT, TAN KAH KEE MRT, SIXTH AV...","151, 154, 156, 157, 170, 170X, 171, 174, 66, 6...",BUKIT TIMAH,CO-ED SCHOOL,MIXED LEVEL,"1.33042992800757,103.804458713785"
1,TEMASEK JUNIOR COLLEGE,22 BEDOK SOUTH ROAD,469278,BEDOK MRT,"12, 16, 31, 38, 47, 137, 155, 196, 197, 229",BEDOK,CO-ED SCHOOL,MIXED LEVEL,"1.31846190599677,103.93559797008899"
2,VICTORIA JUNIOR COLLEGE,20 MARINE VISTA,449035,"Nearest MRT Stations: Kembangan, Eunos","13, 16, 31, 36, 43, 48, 55, 135, 196, 196e, 197",BEDOK,CO-ED SCHOOL,JUNIOR COLLEGE,"1.3057179654943,103.919531033331"
3,EUNOIA JUNIOR COLLEGE,53 MOUNT SINAI ROAD,276880,"Buona Vista MRT Station, Dover MRT Station","92, 100, 105, 106, 111, 147, 185",BISHAN,CO-ED SCHOOL,JUNIOR COLLEGE,"1.3137449647427601,103.786441132957"
4,ANDERSON SERANGOON JUNIOR COLLEGE,4500 ANG MO KIO AVENUE 6,569843,Yio Chu Kang MRT Station,"13, 45, 86, 162, 269, 825, 851, 852, 853",ANG MO KIO,CO-ED SCHOOL,JUNIOR COLLEGE,"1.37839670972544,103.845885968342"


In [224]:
school.to_csv('../data/cleanedschool.csv')

### Cleaning of rooms data

In [304]:
flatroom = pd.read_csv('../data/HDB flat kinds vs rooms.csv').T

In [305]:
flatroom.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, HDB Flat Types to Executive Flat
Data columns (total 2 columns):
0    7 non-null object
1    7 non-null object
dtypes: object(2)
memory usage: 168.0+ bytes


In [306]:
flatroom.columns = flatroom.iloc[0]
flatroom.drop('HDB Flat Types', axis = 0, inplace = True)
flatroom.index = ['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'MULTI-GENERATION', 'EXECUTIVE']
flatroom['RoomType'] = flatroom.index
flatroom

HDB Flat Types,Bedroom,Bathroom,RoomType
2 ROOM,1,1,2 ROOM
3 ROOM,2,2,3 ROOM
4 ROOM,3,2,4 ROOM
5 ROOM,3,2,5 ROOM
MULTI-GENERATION,4,3,MULTI-GENERATION
EXECUTIVE,3,2,EXECUTIVE


In [287]:
flatroom_add = pd.DataFrame({'1 ROOM': {'Bedroom':1, 'Bathroom':1, 'RoomType': '1 ROOM'}}).T
flatroom_add

,Bathroom,Bedroom,RoomType
1 ROOM,1,1,1 ROOM


In [307]:
f = flatroom.append(flatroom_add, ignore_index = True)
                              

In [309]:
f.to_csv('../data/cleanedflatroom.csv')